# San Silvestre 2021

In [1]:
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
from utils import load_data, plot_hist, get_mine, plot_pace
import matplotlib as mpl

In [2]:
mpl.use('TkAgg')
mpl.get_backend()

'TkAgg'

In [3]:
pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(precision=2)
%precision %.2f

'%.2f'

In [4]:

data = load_data()
data.head()

#
data.shape

(16391, 19)

In [5]:
mine = get_mine(data)
mine

,Dorsal,Nombre,Categ.,Puesto,P.Cat.,Tiempo,Ritmo Km.,"Km. 2,5",Km. 5,"Km. 7,5","Tiempo Km. 2,5 (Minutos)",Tiempo Km. 5 (Minutos),"Tiempo Km. 7,5 (Minutos)",Tiempo Km. 10 (Minutos),Ritmo Km. (Minutos),"Ritmo Min/Km. 2,5",Ritmo Min/Km. 5,"Ritmo Min/Km. 7,5",Ritmo Min/Km. 10
5745,31471,Maximiliano Greco,Sénior M,5746,1449,0:52:56,"5'17""/km",0:12:34,0:24:45,0:37:06,12.57,24.75,37.10,52.93,5.28,5.03,4.95,4.95,5.29


# Estadísticas

In [6]:
tiempo_all = data["Tiempo Km. 10 (Minutos)"]
tiempo_mine = mine["Tiempo Km. 10 (Minutos)"]

Cuánto tiempo tardé en completar la carrera?

In [7]:
f"{tiempo_mine.item():.2f} Minutos"

'52.93 Minutos'

De media, cuánto tardaron los corredores en completar la carrera ?

In [8]:
f"{tiempo_all.mean():.2f} Minutos"

'58.15 Minutos'

En qué percetil está mi tiempo ?

In [9]:
f"{stats.percentileofscore(tiempo_all, tiempo_mine.item()):.2f}%"

'35.03%'

Cuánto tardó el corredor mediano ?

In [10]:
f"{tiempo_all.median():.2f} Minutos"

'56.85 Minutos'

Cuánto tardó el corredor que tardó menos que el 80% ?

In [11]:
f"{tiempo_all.quantile(0.2):.2f} Minutos"

'48.75 Minutos'

## Ritmo

In [12]:
ritmos = ["Ritmo Min/Km. 2,5", "Ritmo Min/Km. 5", "Ritmo Min/Km. 7,5", "Ritmo Min/Km. 10"]
mine[ritmos]
data[ritmos].mean()
data[ritmos].std()

Ritmo Min/Km. 2,5   1.67
Ritmo Min/Km. 5     1.27
Ritmo Min/Km. 7,5   1.17
Ritmo Min/Km. 10    1.17
dtype: float64

In [13]:
# ### Categorías
tiempos = ["Km. 2,5 (Minutos)", "Km. 5 (Minutos)", "Km. 7,5 (Minutos)", "Tiempo (Minutos)"]
mine[ritmos].iloc[0]
categ_mine = mine["Categ."].item()
data.query(" `Categ.` == @categ_mine ")
data.groupby("Categ.").mean()[ritmos].sort_values(ritmos[-1])
data.groupby("Categ.").std()[ritmos]

,"Ritmo Min/Km. 2,5",Ritmo Min/Km. 5,"Ritmo Min/Km. 7,5",Ritmo Min/Km. 10
Categ.,,,,
F-35,1.12,1.18,1.07,1.12
F-45,1.03,1.03,1.17,1.12
F-55,1.20,1.29,1.33,1.47
F-65,1.05,1.53,1.63,1.58
Júnior F,0.98,1.03,1.05,1.11
Júnior M,3.54,2.26,1.28,1.17
M-35,1.04,1.00,1.02,1.06
M-45,1.78,1.07,1.10,1.10
M-55,1.00,1.06,1.03,1.08


Andando

In [14]:
len(data[data["Ritmo Min/Km. 10"] > 13])

27

# Plot

plt.ion()
plot_hist(
    "Tiempo (Minutos)",
    data,
    title="Distribución del Tiempo en minutos",
    xlabel="Minutos",
    ylabel="%",
)
plot_hist(
    "Ritmo Km. (Minutos)",
    data,
    title="Distribución del Ritmo ",
    xlabel="Min/Km",
)
plot_hist("Km. 2,5 (Minutos)", data, title="")
plot_hist("Km. 5 (Minutos)", data, title="")
plot_hist("Km. 7,5 (Minutos)", data, title="")

Cual fue la evolución de los corredores por cada etapa?

delta1 = data["Km. 5 (Minutos)"] - data["Km. 2,5 (Minutos)"]
delta2 = data["Km. 7,5 (Minutos)"] - data["Km. 5 (Minutos)"]
delta3 = data["Tiempo (Minutos)"] - data["Km. 7,5 (Minutos)"]

delta1.div(2.5).hist(bins=20, alpha=0.5)
delta2.div(2.5).hist(bins=20, alpha=0.5)
delta3.div(2.5).hist(bins=20, alpha=0.5)

plt.show()


plot_pace(data)